# 10. Gépi tanulás – gyakorlat

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## 1. Scikit-learn

### 1.1. Python csomagok gépi tanuláshoz

* [numpy](http://www.numpy.org/): matematikai számítások, lineáris algebra
* [scipy](https://www.scipy.org/): numpy-ra épülő, tudományos számítások
* [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/): adatvizualizáció, plottolás
* [pandas](http://pandas.pydata.org/): dataframe adatszerkezet adatok kezeléséhez
* [scikit-learn](http://scikit-learn.org/stable/): gépitanulás

### 1.2. Scikit-learn bevezető

Várt adatok:

* feature-mátrix:
    * egy sor = egy példány
    * egy oszlop = egy feature
    * általában *X*-el jelölik
    * Numpy *array* vagy pandas *DataFrame* típusúban kell megadni
* target-vektor:
    * egy sor = egy példány, minden eleme megfelel a *feature-mátrix* egy sorának
    * általában *y*-nal jelölik
    * Numpy *array* vagy pandas *Series* tipusú lehet
    * csak a felügyelt tanuláshoz kell

Lépések:

1. adatok előkészítése (X, y, train, test)
2. módszer választása: `form sklearn.xyz_modul import választott_módszer`
3. modell létrehozása: `model = valasztott_modszer(paraméterek...)`
4. tanítás: `model.fit(X, y)`
5. modell használata: `model.predict(new_X)`
6. kiértékelés (ha van target-vektorunk *new_X*-hez): ehhez az `sklearn.metrics`-ben vannak hasznos dolgok 


### 1.3. Példa

Gépi tanulós *hello world*: iriszek három osztályba sorolása levélméreteik alapján.

<img src="https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg" />

**jegyek**: *sepal_length*, *sepal_width*, *petal_length*, *petal_width*

**címkék**: *setosa*, *versicolor*, *virginica*

#### 1. adatok előkészítése

In [ ]:
# adatkupac beszerzése
import seaborn as sb
iris = sb.load_dataset('iris')
print(iris.__class__.__name__)
print(iris.shape)
print(iris.head(5))

In [ ]:
# feature-mátrix
X = iris.drop('species', axis=1)
print(X.head())

In [ ]:
# tareget-vektor
y = iris['species']
print(y.head())

In [ ]:
# adatok vágása tanításhoz és teszteléshez
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
print(len(X_train), len(y_train), len(X_test), len(y_test))

#### 2. módszer választása

Naiv Bayes módszerek:

* egyszerűek, nem nagyon kell paraméterezni
* gyorsak
* feltevés, hogy a jegyek függetlenek egymástól

In [ ]:
from sklearn.naive_bayes import GaussianNB

#### 3. modell létrehozása

In [ ]:
model = GaussianNB() # itt lehetnek további megadandó paraméterek az iniciálizáláshoz, de most nincsenek

#### 4. tanítás

In [ ]:
model.fit(X_train, y_train)

#### 5. jóslás

In [ ]:
y_predicted = model.predict(X_test)

#### 6. kiértékelés

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print('accurcy: ', accuracy_score(y_test, y_predicted))
print('\nREPORT:')
print(classification_report(y_test, y_predicted))

In [ ]:
# confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

confmat = confusion_matrix(y_test, y_predicted)
sb.heatmap(confmat, square=True, annot=True, cbar=False, 
           xticklabels=['setosa', 'versicolor', 'virginica'], 
           yticklabels=['setosa', 'versicolor', 'virginica'])
plt.xlabel('predicted value')
plt.ylabel('true value')

### 1.4. Kitekintés

* [felügyelt módszerek](http://scikit-learn.org/stable/supervised_learning.html)
* [jegykinyerés szövegből](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
* [grid search](http://scikit-learn.org/stable/modules/grid_search.html#exhaustive-grid-search)
* [pipeline](http://scikit-learn.org/stable/modules/pipeline.html)

## 2. Huntag3

### 2.1. Előkészületek

* [HunTag3](https://github.com/ppke-nlpg/HunTag3) letöltése
* kitömörítése

In [ ]:
%%bash

wget -q -c https://github.com/ppke-nlpg/HunTag3/archive/master.zip
unzip -q -u master.zip

### 2.2. Adatok

Forrás: http://hlt.sztaki.hu/resources/hunnerwiki.html

Oszlopok:

1. **token**
2. egyértelműsített morfológiai elemzés (KR kód)
3. lemma
4. **gold standard**

In [ ]:
!head -n 13 huwiki.ner.sample.train.tsv

### 2.3. Features

A *feature*-öket *on the fly* generáljuk a tanítás során rövid függvényekkel, melyek bemenetüket a TSV valamelyik oszlopából vehetik.

Ezen függvények helye a *features.py* fájlban van. A HunTag3-mal jön egy alap *features.py*, ezt módosíthatjuk, vagy sajátot is írhatunk.

Hogy milyen featur-öket akarunk használni, és melyik feature a TSV melyik oszlopát használja, azt az általunk írt konfigfájlban tudjuk megadni.

#### konfig

A konfigurációs beállításokat [YAML](https://en.wikipedia.org/wiki/YAML) formátumban kell megadni.

* *name*: a *most-informative-features* kimenetében ezen a néven jelenik meg a *feature*
* *type*: lehet *token*, *sentence*, vagy *lex*, ezek a feature-ök függvényeinek is prefixei egyben
* *actionName*: a függvény neve, *type* prefix nélkül
    * *lex*: lexikai jegynél a lexikon elérési útját kell megadni
* *fields*: TSV oszlop száma (0-ával kezdődik)

In [ ]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
# - # org lista
#    name: orglist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/org.fromtraincorpus.lex
#    fields: 0
# - # loc lista
#    name: loclist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/loc.fromtraincorpus.lex
#    fields: 0
# - # misc lista
#    name: misclist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/misc.fromtraincorpus.lex
#    fields: 0
# - # per lista
#    name: perlist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/per.fromtraincorpus.lex
#    fields: 0
...



#### features

* egy feature-függvénynek listával kell visszatérnie
* a *bool* típusú visszatérési értékeket konvertálni kell *int*-re

In [ ]:
%%writefile HunTag3-master/features.py

def token_isUpper(token, _=None):
    return [int(token.isupper())]

def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]


### 2.4. Tanítás

#### megfigyelési modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

#### átmenet modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

### 2.5. Címkézés, tesztelés, kiértékelés

### címkézés

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

#### kiértékelés

In [ ]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -m BI -i huwiki.ner.test.tagged.tsv

### 2.6. Feladatok

Implementáljatok pár új feature-t és értékeljétek ki ezekkel a ner-taggert.
Az új feature-ök legyenek:

* szófaj (része az elemzésnek a 2. oszlopban)
* lemma (3. oszlop)
* nagybetűvel kezdődik-e
* van-e benne nagybetű

#### konfig

In [ ]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
# - # szófajkód
#    name: pos
#    type: ...
#    actionName: getPos
#    fields: ...
#
# - # lemma
#    name: lem
#    type: ...
#    actionName: getLem
#    fields: ...
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
# - # nagybetűvel kezdődik-e
#    name: isTitle
#    type: ...
#    actionName: isTitle
#    fields: ...
# - # van-e benne nagybetű
#    name: hasUpper
#    type: ...
#    actionName: hasUpper
#    fields: ...
...



#### features

In [ ]:
%%writefile HunTag3-master/features.py


def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]

#def token_getPos(ana, _=None):
#    return ...

#def token_getLem(lem, _=None):
#    return ...

def token_isUpper(token, _=None):
    return [int(token.isupper())]

#def token_isTitle(token, _=None):
#    return ...

#def token_hasUpper(token, _=None):
#    return ...



#### megfigyelési modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

#### átmenet modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

#### címkézés

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

#### kiértékelés

In [ ]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -m BI -i huwiki.ner.test.tagged.tsv